# Extracting segments

In this notebook, we will trim the context down to shorter segments containing the drug and AE

In [16]:
import pandas as pd
import re

In [17]:
# function to replace symbols and acronyms
def replace_line_break(text):
    text = text.replace("  .  "," - ")
    text = text.replace("  "," - ")
    return text


In [18]:
# import dataset
data = pd.read_excel (r'.\train dataset.xlsx')

In [ ]:
data.head()

In [20]:
data['context2']=data['context'].apply(lambda x : replace_line_break(x))

In [21]:
contexts=data['context2'] # user context with line breaks replaced for the segmentation
aes=data['ae']
drugs=data['drug']

In [ ]:
#procedure to extract segments from context

segments=[]
min_drug=[]
max_drug=[]
min_ae=[]
max_ae=[]
for i in range(len(contexts)):
    context=contexts[i]
    ae=aes[i].strip()
    drug=drugs[i].strip()
    split=re.split(r'[.,)/-]',context) # for each context, we break it up where there are symbols, e.g. "." "," ")" "-" 
    min_index_drug=0
    drug_found=0
    max_index_drug=0
    min_index_ae=0
    ae_found=0
    max_index_ae=0
    for i in range(len(split)):
        if drug in split[i]: # if the drug name is found in the segment
            if drug_found==0: # and the drug has not yet been found
                drug_found=1 # update the drug_found flag
                min_index_drug=i # update the min_index
            max_index_drug=i # update the max_index
        if ae in split[i]:
            if ae_found==0: # and the drug has not yet been found
                ae_found=1 # update the drug_found flag
                min_index_ae=i # update the min_index
            max_index_ae=i # update the max_index
    min_index= min(min_index_drug,min_index_ae) #identify the first index where ae or drug occurs
    max_index= max(max_index_drug,max_index_ae) #identify the first or last index where ae or drug occurs
    segment=''
    for j in range(min_index,max_index+1):
        segment = segment+' '+split[j] # join these segments to get the final segment
    print(split)
    print(ae)
    print(drug)
    print(segment)
    segments.append(segment)
    min_drug.append(min_index_drug)
    max_drug.append(max_index_drug)
    min_ae.append(min_index_ae)
    max_ae.append(max_index_ae)


In [23]:
# add on additional fields
data['min_drug']=min_drug
data['max_drug']=max_drug
data['min_ae']=min_ae
data['max_ae']=max_ae
data['segment']=segments

data.to_csv('./train_segmented.csv')

In [ ]:
data